In [2]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 경고 뜨지 않게
import warnings
warnings.filterwarnings('ignore')

#저장라이브러리
import pickle

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# LabelEncoder
from sklearn.preprocessing import LabelEncoder

# 데이터를 학습용과 검증용으로 나눈다.
from sklearn.model_selection import train_test_split

import os


# 딥러닝
import tensorflow as tf

# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층구조를 정의하는 것
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation

# 다중 분류를 위한 원핫 인코딩
from tensorflow.keras.utils import to_categorical

# 저장된 학습 모델 복원
from tensorflow.keras.models import load_model

# epoch마다 모델을 저장하는 함수
from tensorflow.keras.callbacks import ModelCheckpoint
# 조기 중단
from tensorflow.keras.callbacks import EarlyStopping

# 출력한 것을 청소하는 함수
from IPython.display import clear_output
# 시간 관련
import time

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [4]:
# 랜덤시드를 설정한다.
np.random.seed(1)
tf.random.set_seed(1)

In [6]:
# 데이터를 불러온다.
df1 = pd.read_csv('data/wine.csv', header=None)
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [8]:
# 입력과 결과로 나눈다.
X = df1.drop(12, axis=1)
y = df1[12]

display(X)
display(y)

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


0       1
1       1
2       1
3       1
4       1
       ..
6492    0
6493    0
6494    0
6495    0
6496    0
Name: 12, Length: 6497, dtype: int64

In [10]:
# 모델 설정
model = Sequential()

model.add(Dense(16, input_dim=12))
model.add(Activation('relu'))

model.add(Dense(12))
model.add(Activation('relu'))

model.add(Dense(8))
model.add(Activation('relu'))

model.add(Dense(4))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [12]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [14]:
# 학습 모델들이 저장될 위치
path1 = 'models/20_와인 분류 - 조기중단/최종.h5'
# 자동 저장 콜백
call1 = ModelCheckpoint(filepath=path1, monitor='val_loss', save_best_only=True)

In [18]:
# 조기 중단
# patience : 현재를 기준으로 몇회를 더 확인할 것인가.
# 현재의 오차가 지금까지의 오차보다 가장 낮은 수치를 보였다면 지금부터 patience 만큼
# 더 확인해 본다. 그 안에 오차가 더 작아지면 그 때부터 다시 patience 만큼을 확인한다.
# 만약 patience 만큼 더 살펴보았는데 더 이상 개선되지 않으면 학습이 중단된다.
call2 = EarlyStopping(monitor='val_loss', patience=10)

In [20]:
# 학습한다.
model.fit(X, y, epochs=2000, batch_size=7000, validation_split=0.2, 
         callbacks=[call1, call2])

Epoch 1/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6750 - loss: 0.6932

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.6750 - loss: 0.6932 - val_accuracy: 0.9969 - val_loss: 0.6926
Epoch 2/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.6875 - loss: 0.6930

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.6875 - loss: 0.6930 - val_accuracy: 1.0000 - val_loss: 0.6921
Epoch 3/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.6921 - loss: 0.6928

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - accuracy: 0.6921 - loss: 0.6928 - val_accuracy: 1.0000 - val_loss: 0.6916
Epoch 4/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.6923 - loss: 0.6926

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step - accuracy: 0.6923 - loss: 0.6926 - val_accuracy: 1.0000 - val_loss: 0.6911
Epoch 5/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.6923 - loss: 0.6924

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 600ms/step - accuracy: 0.6923 - loss: 0.6172 - val_accuracy: 1.0000 - val_loss: 0.3693
Epoch 1997/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.6923 - loss: 0.6172

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step - accuracy: 0.6923 - loss: 0.6172 - val_accuracy: 1.0000 - val_loss: 0.3693
Epoch 1998/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.6923 - loss: 0.6172

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.6923 - loss: 0.6172 - val_accuracy: 1.0000 - val_loss: 0.3693
Epoch 1999/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.6923 - loss: 0.6172

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.6923 - loss: 0.6172 - val_accuracy: 1.0000 - val_loss: 0.3693
Epoch 2000/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.6923 - loss: 0.6172

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step - accuracy: 0.6923 - loss: 0.6172 - val_accuracy: 1.0000 - val_loss: 0.3693
